In [2]:
import autogen

from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from rag_teachablitity.teachability import Teachability

import chromadb

import nest_asyncio
nest_asyncio.apply()


config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location="..",
    filter_dict={
        "model": ["gpt-3.5-turbo", "gpt-35-turbo", "gpt-35-turbo-0613", "gpt-4", "gpt4"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)



LLM models:  ['gpt-4']


In [3]:
# Configuration for the Language Model (LLM)
llm_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    "seed": 42,
}

# Configuration for the manager using the same config_list as llm_config
manager_config = {
    "config_list": config_list,  # config_list should be defined or imported
    "timeout": 60,
    "seed": 42,
}


In [6]:
import asyncio
from typing import Dict, List, Optional, Union, Callable
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.formatting_utils import colored
from typing_extensions import Annotated

import arxiv

db_dir = './teachability_db'
# check if db_dir exists, delete it if it does
import os
import shutil
if os.path.exists(db_dir): shutil.rmtree(db_dir)

def retrieve_content(query_text: Annotated[str, "The list of query texts to search for."], 
                    n_results: Annotated[int, "The number of results to retrieve for each query."] = 3,
                    ) -> str:
    
    # Start by instantiating any agent that inherits from ConversableAgent.
    teachable_agent = autogen.ConversableAgent(
        name="teachable_agent",  # The name is flexible, but should not contain spaces to work in group chat.
        llm_config={"config_list": config_list, "timeout": 120, "cache_seed": None},  # Disable caching.
    )

    # Instantiate the Teachability capability. Its parameters are all optional.
    teachability = Teachability(
        verbosity=0,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
        reset_db=False,  
        path_to_db_dir=db_dir,
        recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
    )

    # Now add the Teachability capability to the agent.
    teachability.add_to_agent(teachable_agent)

    # Instantiate a UserProxyAgent to represent the user. But in this notebook, all user input will be simulated.
    user = autogen.UserProxyAgent(
        name="user",
        human_input_mode="NEVER",
        is_termination_msg=lambda x: "TERMINATE" in x.get("content"),
        max_consecutive_auto_reply=0,
        code_execution_config={"use_docker": False},
    )

    sort_by = arxiv.SortCriterion.Relevance
    papers = arxiv.Search(
        query=query_text,
        max_results=n_results,
        sort_by=sort_by
        )

    for paper in arxiv.Client().results(papers):
        user.initiate_chat(teachable_agent,
                           message=f"The following article is one of the article that I found for '{query_text}' topic: /n/n '{paper.title}' by {paper.authors} updated on {paper.updated}: {paper.pdf_url} \nsummery: {paper.summary} \n?")

    results = list(arxiv.Client().results(papers))
    return "to PI: the database is updated, go ahead and retrieve the topics you are interested in from database. if something is missing, let me know and I will update the database for you."
    # return "Context is:" + '/n'.join([f"{paper.title} by {paper.authors} updated on {paper.updated}: {paper.pdf_url} \n{paper.summary} \n" for paper in results])


message = "Overview of time series forecasting methods"
retrieve_content(message, n_results=3)

user (to teachable_agent):

The following article is one of the article that I found for 'Overview of time series forecasting methods' topic: /n/n 'Inter Time Series Sales Forecasting' by [arxiv.Result.Author('Manisha Gahirwal')] updated on 2013-03-01 09:06:03+00:00: http://arxiv.org/pdf/1303.0117v1 
summery: Combining forecast from different models has shown to perform better than
single forecast in most time series. To improve the quality of forecast we can
go for combining forecast. We study the effect of decomposing a series into
multiple components and performing forecasts on each component separately...
The original series is decomposed into trend, seasonality and an irregular
component for each series. The statistical methods such as ARIMA, Holt-Winter
have been used to forecast these components. In this paper we focus on how the
best models of one series can be applied to similar frequency pattern series
for forecasting using association mining. The proposed method forecast val

KeyboardInterrupt: 

In [5]:
# System message constants for different roles

COORDINATOR = """You are a Research coordinator: This is the person who coordinates the various aspects of the research project. 
The role is also responsible to rephrase research questions into key word queries for the arxiv api. 
if researcher or critic needs help, you can help them to find the information from the arxiv API.
"""
RESEARCHER = """You are a Researcher: This is the person who performs the research of paper's summaries. 
You MUST take to account the feedback from the critic to improve the quality of the work.
In your role, you have the autonomy to question the provided content and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager.
"""
SUB1 = """You are a Sub-investigator (Sub-I): This is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics.
you MUST help PI to ensure that the research team is on the right track and the research is going in the right direction. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
Reply 'TERMINATE' in the end when everything is done.
"""

# If there isn't enough information below, you should reply exactly 'UPDATE CONTEXT'.
QNA_PROMPT = """Assistant helps the researchers with searching information from the arxiv API. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. Do not generate answers that don't use the sources below. 
In your role, you have the autonomy to question the provided content or the process presented in this group chat and can request corrections or seek clarification if there is something that appears to be missing or unclear after executing a given task. If at any point you find yourself confused or in need of assistance, do not hesitate to reach out to the group chat manager, who can guide you or delegate the task to another qualified participant.
For tabular information return it as an html table. Do not return markdown format. If the question is not in English, answer in the language used in the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, for example [info1.txt]. Don't combine sources, list each source separately.
User's question is: {input_question}

Context is: {input_context}"""

# Termination message definition
termination_msg = (
    lambda x: isinstance(x, dict)
    and str(x.get("content", "")).upper() == "TERMINATE"
)

# Agent definitions
principalInvestigator = autogen.UserProxyAgent(
    name="PI",
    is_termination_msg=termination_msg,
    human_input_mode="TERMINATE",
    system_message="""You are a Principal investigator (PI): You are the leader of the research team who asks the questions and gives task.
    You MUST make sure that the research team is on the right track and the research is going in the right direction. 
    You should check the work of the researcher, coordinator and Planner and provide feedback to improve the quality of the work or confirm the work.
    reply 'TERMINATE' in the end when everything is done.
    """,
    llm_config=llm_config, 
    code_execution_config=False,
    description="Principal investigator (PI) is the leader of the research team who asks the questions and gives task."
)

PLANNER = """Planner. Suggest a plan. Revise the plan based on feedback from PI and critic, until PI approval.
The plan may involve a Research Coordinator to to rephrase research questions into key word queries for the arxiv api and a Researcher who could performs the research of paper's summaries. 
Explain the plan first. Be clear which step is performed by an Research Coordinator, and which step is performed by a Researcher.
"""

planner = autogen.AssistantAgent(
    name="Planner",
    system_message=PLANNER,
    llm_config=llm_config,
    description="Planner suggests a plan and revises the plan based on feedback from PI and critic, until PI approval."
)

subInvestigator = autogen.AssistantAgent(
    name="Sub-I",
    is_termination_msg=termination_msg,
    system_message=SUB1,
    llm_config=llm_config,
    description="Sub-investigator (Sub-I) is the assistant to the PI, who helps with the tasks of the PI with a step wise research plan with sub-research topics."
)

critic = autogen.AssistantAgent(
    name="Critic",
    system_message="Critic. Double check the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work",
    llm_config=llm_config,
    description="Critic is responsible for double checking the work of researcher, research coordinator and Planner and provide feedback to improve the quality of the work"
)


researchCoordinator = autogen.AssistantAgent(
    name="ResearchCoordinator",
    is_termination_msg=termination_msg,
    system_message=COORDINATOR,  # COORDINATOR should be a predefined string variable
    llm_config=llm_config,
    description="Research coordinator is the person who rephrase research questions into key word queries for the arxiv api."
)

# create a UserProxyAgent instance named "user_proxy"
RC_proxy = autogen.UserProxyAgent(
    name="ResearchCoordinator_proxy",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: "content" in x
    and x["content"] is not None
    and x["content"].rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "ResearchCoordinator",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    description="assist Research coordinator to query for the arxiv api."
)

researcher = autogen.AssistantAgent(
    name="Researcher",
    is_termination_msg=termination_msg,
    system_message=RESEARCHER, 
    llm_config=llm_config,
    description="Researcher is the person who performs the research of paper's summaries."
)


from chromadb.config import Settings

db_dir = './teachability_db_AI_safety'
settings = Settings(
            anonymized_telemetry=False, allow_reset=True, is_persistent=True, persist_directory=db_dir
        )

Researcher_aid = RetrieveUserProxyAgent(
    name="RAG_USER",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    llm_config=llm_config, 
    retrieve_config={
        "task": "code",
        "customized_prompt": QNA_PROMPT, 
        "docs_path": None,
        "model": config_list[0]["model"],
        "client": chromadb.Client(settings),
        "collection_name": "memos",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
    description="Assistant has extra content retrieval power and can provide team members with context in database.",
)

# # Instantiate the Teachability capability. Its parameters are all optional.
# teachability = Teachability(
#     verbosity=3,  # 0 for basic info, 1 to add memory operations, 2 for analyzer messages, 3 for memo lists.
#     reset_db=False,
#     path_to_db_dir=db_dir,
#     recall_threshold=1.5,  # Higher numbers allow more (but less relevant) memos to be recalled.
# )

# # Now add the Teachability capability to the agent.
# teachability.add_to_agent(researcher)


In [ ]:
def custom_speaker_selection_func(last_speaker: Agent, groupchat: autogen.GroupChat):
    """Define a customized speaker selection function.
    A recommended way is to define a transition for each speaker in the groupchat.

    Returns:
        Return an `Agent` class or a string from ['auto', 'manual', 'random', 'round_robin'] to select a default method to use.
    """
    messages = groupchat.messages

    if len(messages) <= 1:
        # first, let the engineer retrieve relevant data
        return planner

    if last_speaker is planner:
        # if the last message is from planner, let the engineer to write code
        return engineer
    elif last_speaker is user_proxy:
        if messages[-1]["content"].strip() != "":
            # If the last message is from user and is not empty, let the writer to continue
            return writer

    elif last_speaker is engineer:
        if "```python" in messages[-1]["content"]:
            # If the last message is a python code block, let the executor to speak
            return executor
        else:
            # Otherwise, let the engineer to continue
            return engineer

    elif last_speaker is executor:
        if "exitcode: 1" in messages[-1]["content"]:
            # If the last message indicates an error, let the engineer to improve the code
            return engineer
        else:
            # Otherwise, let the writer to speak
            return writer

    elif last_speaker is writer:
        # Always let the user to speak after the writer
        return user_proxy

    else:
        # default to auto speaker selection method
        return "auto"



In [5]:
def _reset_agents():
    principalInvestigator.reset()
    subInvestigator.reset()
    researchCoordinator.reset()
    researcher.reset()
    planner.reset()

def call_rag_chat(PROBLEM):
    _reset_agents()  # Resets the state of all the agents before starting the chat

    autogen.agentchat.register_function(
        retrieve_content,
        caller=researchCoordinator,
        executor=RC_proxy,
        name="arvix_retrieve_content",
        description="Retrieve content for code question answering."
    )

    # Create the GroupChat manager instance.
    groupchat = autogen.GroupChat(
        # agents=[principalInvestigator, planner, researchCoordinator, researcher, critic, RC_proxy, Researcher_aid],
        agents=[principalInvestigator, planner, researcher, critic, Researcher_aid],
        messages=[],
        max_round=35,
        speaker_selection_method="auto",
        allow_repeat_speaker=False,
    )

    manager = autogen.GroupChatManager(
        groupchat=groupchat,
        llm_config=manager_config,
    )

    # Initialize the chat with the primary investigator as the proxy agent.
    principalInvestigator.initiate_chat(
        manager,
        message=PROBLEM
    )

    # Start chatting with the primary investigator acting as the user proxy agent.
    return principalInvestigator.chat_messages

# Example usage:
# You MUST Collect enough information from the arxiv API and summerize the information in the blog post.
PROBLEM = """Write blog post about the modelling of reliability and safety mechanisms in AI system. 
The focus MUST be on Large Language Models.
You MUST first plan the tasks for the research team.
You MUST relay only the information in your database.
The blog post MUST be written in a way that is easy to understand for a non-technical audience.
The blog post MUST be up to date and include the latest research in the field in your database.
"""
messages = call_rag_chat(PROBLEM)
print(messages)


PI (to chat_manager):

Write blog post about the modelling of reliability and safety mechanisms in AI system. 
The focus MUST be on Large Language Models.
You MUST first plan the tasks for the research team.
You MUST relay only the information in your database.
The blog post MUST be written in a way that is easy to understand for a non-technical audience.
The blog post MUST be up to date and include the latest research in the field in your database.


--------------------------------------------------------------------------------


Planner (to chat_manager):

Initial Plan:

1. Define the Objectives:
   - The primary goal is to write an informative and easy-to-understand blog post about the modeling of reliability and safety mechanisms in large language models (LLMs).

2. Preliminary Research:
   - The Research Coordinator (RC) will initiate a literature review to identify key topics and research questions that need to be addressed concerning reliability and safety in LLMs.

3. Synthesize Research Questions into Keywords:
   - The RC will translate the research questions into relevant keywords and search queries. For example, "reliability in large language models," "safety mechanisms in AI," "risk mitigation LLM," etc.

4. Search for Relevant Literature:
   - The RC will use the identified keywords to query the arXiv API for the latest research papers and studies relevant to the topic.

5. Compile Research Material:
   - The RC will compile a list of the retrieved papers along with their summaries and abstracts an

KeyboardInterrupt: 

In [ ]:
# Example usage:
PROBLEM = "Write a comperhansive blog post about the modelling of reliability and safety mechanisms in AI system. The focus should be on Large Language Models."
messages = call_rag_chat(PROBLEM)
print(messages)

PI (to chat_manager):

Write a comperhansive blog post about the modelling of reliability and safety mechanisms in AI system. The focus should be on Large Language Models.

--------------------------------------------------------------------------------


Planner (to chat_manager):

Plan for Composing a Blog Post on the Modeling of Reliability and Safety Mechanisms in AI Systems, Focusing on Large Language Models:

1. **Initial Research Coordination:**
   - **Research Coordinator Task:** Formulate research questions by framing the key issues surrounding reliability and safety in AI systems, with a focus on Large Language Models (LLMs). Examples of research questions include:
       - What are the latest methods for ensuring the reliability of Large Language Models?
       - How are safety mechanisms currently being integrated into AI systems?
       - What challenges arise in modeling reliability and safety for LLMs?
   - **Research Coordinator Task:** Convert these research questions into keyword queries suitable for an API search, such as "reliability in large language models," "AI safety mechanisms," "error mitigation in AI," etc.

2. **Initial Research Process:**
   - **Researcher Task:** Utilize the arXiv API to run the keyword que

KeyboardInterrupt: 